In [1]:
import numpy as np
import pandas as pd
import requests
from bs4 import BeautifulSoup
from tqdm import tqdm
from selenium import webdriver
import time
from collections import defaultdict
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import re

from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import NoSuchElementException, TimeoutException, WebDriverException

In [3]:
driver = webdriver.Chrome()

In [5]:
driver.get('https://www.linkedin.com/login?fromSignIn=true&trk=guest_homepage-basic_nav-header-signin')

In [7]:
import random
import time

my_mail = "xyz@gmail.com"
password = 'xyz'
# username = "xyz"

wait = WebDriverWait(driver, 10)

time.sleep(random.uniform(1.5, 4))


input_box = wait.until(EC.visibility_of_element_located((
    By.CSS_SELECTOR, "input[id='username']"
)))
input_box.clear()
input_box.send_keys(my_mail)

time.sleep(random.uniform(1.5, 5))

password_box = wait.until(EC.visibility_of_element_located((
    By.CSS_SELECTOR, "input[id='password']"
)))
password_box.clear()
password_box.send_keys(password)

time.sleep(random.uniform(4, 6))


sign_in_button = wait.until(EC.element_to_be_clickable((
    By.CSS_SELECTOR, "button[aria-label='Sign in']"
)))
sign_in_button.click()


In [8]:
from datetime import datetime, timedelta

def relative_to_date(text):
    today = datetime.today()
    
    if 'h' in text:
        hours = int(text.replace('h', ''))
        return today - timedelta(hours=hours)
    elif 'd' in text:
        days = int(text.replace('d', ''))
        return today - timedelta(days=days)
    elif 'w' in text:
        weeks = int(text.replace('w', ''))
        return today - timedelta(weeks=weeks)
    elif 'mo' in text:
        months = int(text.replace('mo', ''))
        return today - timedelta(days=30 * months)
    elif 'y' in text:
        years = int(text.replace('y', ''))
        return today - timedelta(days=365 * years)
    else:
        return today  # fallback

print(relative_to_date("2d"))


2025-06-21 17:43:28.062889


In [9]:
blinkit_keywords = [

    # 📦 Service / Delivery Issues
    "Blinkit Complaint",
    "Blinkit late delivery",
    "Blinkit delivery issue",
    "Blinkit order not delivered",
    "Blinkit delivery delay",
    "Blinkit missed delivery",
    "Blinkit wrong item",
    "Blinkit delivery boy issue",
    "Blinkit cancelled my order",
    "Blinkit not delivering",
    "Blinkit didn't deliver",
    "Blinkit delivery scam",
    "Blinkit service complaint",

    # 💳 Refund / Payment Issues
    "Blinkit refund not received",
    "Blinkit refund delay",
    "Blinkit charged twice",
    "Blinkit payment failed",
    "Blinkit payment issue",
    "Blinkit not refunding",
    "Blinkit money not returned",
    "Blinkit deducted money",

    # 🥦 Product Quality Issues
    "Blinkit expired product",
    "Blinkit rotten vegetables",
    "Blinkit damaged item",
    "Blinkit sent stale food",
    "Blinkit poor packaging",
    "Blinkit quality issue",
    "Blinkit not fresh",

    # 👎 Negative Sentiment / Experience
    "Blinkit worst experience",
    "Blinkit terrible service",
    "Blinkit poor customer service",
    "Blinkit very bad experience",
    "Blinkit never ordering again",
    "Blinkit worst app ever",
    "Blinkit horrible service",
    "Blinkit frustrating",
    "Blinkit disappointed",
    "Blinkit cheated",
    "Blinkit scam",
    "Blinkit fraud",

    # 🧑‍💻 Customer Support Issues
    "Blinkit no response",
    "Blinkit not responding",
    "Blinkit support not working",
    "Blinkit customer care",
    "Blinkit help",
    "Blinkit complaint",
    "Blinkit issue"
]

In [ ]:
import time
import random
import re
from bs4 import BeautifulSoup
from tqdm import tqdm
from datetime import datetime, timedelta

all_posts = set()
post_list = []


for query in tqdm(blinkit_keywords):
    wr_query = ("%20").join(query.split())
    link = "https://www.linkedin.com/search/results/content/?datePosted=%22past-month%22&keywords=" + wr_query+ "&origin=FACETED_SEARCH&sid=GtM"


    driver.get(link)
    time.sleep(random.uniform(4, 7))
    
    last_height = driver.execute_script("return document.body.scrollHeight")
    
    while True:
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(random.uniform(4, 7))
    
        soup = BeautifulSoup(driver.page_source, 'html.parser')
        posts = soup.find_all('div', class_='update-components-actor__meta')  # reliable anchor
        posts_full = soup.find_all('div', attrs={"data-urn": True, "role": "article"})
    
        for post in posts_full:
            try:
                p1 = post.find('div', class_="fie-impression-container").find_all('div', recursive=False)
            except Exception as e:
                print("Skipped post - outer structure missing:", e)
                continue
    
            try:
                check_id = post.get('data-urn')
                post_link = "https://www.linkedin.com/feed/update/" + check_id
            except:
                post_link = ""
    
            if post_link in all_posts:
                continue
    
            try:
                user_profile = p1[0].find('a').get('href')
            except:
                user_profile = ""
    
            try:
                name = p1[0].find('span', class_="update-components-actor__title")\
                           .find('span', dir='ltr')\
                           .find('span', class_=None).text.strip()
            except:
                name = ""
    
            try:
                spans = p1[0].find_all("span", {"aria-hidden": "true"})
                initial_date = ""
                for span in spans:
                    txt = span.get_text(strip=True)
                    match = re.match(r"(\d+)(w|d|mo|h|y)", txt)
                    if match:
                        initial_date = match.group(0)
                        break
    
                if initial_date:
                    date_post = relative_to_date(initial_date).strftime('%Y-%m-%d')
                else:
                    date_post = ""
            except:
                date_post = ""
    
            try:
                post_text = p1[1].find('span', dir="ltr").text.strip()
            except:
                post_text = ""
    
            try:
                if post.find('button', class_="update-components-image__image-link") and \
                   post.find('button', class_="update-components-image__image-link").find('img'):
                    have_img_vid = "Yes"
                    p_stats = p1[3]
                else:
                    raise Exception
            except:
                have_img_vid = "No"
                try:
                    p_stats = p1[2]
                except:
                    p_stats = None
    
            try:
                likes = p_stats.find('ul', class_="display-flex flex-wrap")\
                               .find_all('li')[0].find('span').text.strip()
            except:
                likes = "0"
    
            try:
                comments = p_stats.find('ul', class_="display-flex flex-wrap")\
                                  .find_all('li')[1]\
                                  .find('ul', class_="display-flex flex-grow-1 max-full-width")\
                                  .find_all('li')[0].text.strip().split(" ")[0]
            except:
                comments = "0"
    
            try:
                reposts = p_stats.find('ul', class_="display-flex flex-wrap")\
                                 .find_all('li')[1]\
                                 .find('ul', class_="display-flex flex-grow-1 max-full-width")\
                                 .find_all('li')[1].text.strip().split(" ")[0]
            except:
                reposts = "0"
    
            print(check_id)
            
            post_list.append([
                name,
                query,
                user_profile,
                date_post,
                post_text,
                post_link,
                have_img_vid,
                comments,
                reposts,
                likes            
            ])
    
            all_posts.add(post_link)
            
        new_height = driver.execute_script("return document.body.scrollHeight")
        if new_height == last_height:
            break
        last_height = new_height
    
        time.sleep(random.uniform(3, 5))


In [15]:
len(post_list)

1612

In [17]:
len(all_posts)

1612

In [19]:
columns = [
    "Name",
    "Query",
    "Profile Link",
    "Date Posted",
    "Post Text",
    "Post Link",
    "Media",
    "Comments",
    "Reposts",
    "Likes"
]

df = pd.DataFrame(post_list, columns=columns)
df.head()

,Name,Query,Profile Link,Date Posted,Post Text,Post Link,Media,Comments,Reposts,Likes
0,Rahul Mishra,Blinkit Complaint,https://www.linkedin.com/in/rahul-mishra-aab44...,2025-06-17,No one wants to talk about it.But every manufa...,https://www.linkedin.com/feed/update/urn:li:ac...,Yes,0,0,4
1,Ali Ahmad,Blinkit Complaint,https://www.linkedin.com/in/aliahmed292?miniPr...,2025-06-17,hashtag#grocery hashtag#retail hashtag#quickco...,https://www.linkedin.com/feed/update/urn:li:ac...,No,40,1,107
2,Harshit Joshi,Blinkit Complaint,https://www.linkedin.com/in/harshitjoshi001?mi...,2025-06-17,You may get delivery in 10 mins. But brand lik...,https://www.linkedin.com/feed/update/urn:li:ac...,Yes,9,0,97
3,Vinni Aggarwal,Blinkit Complaint,https://www.linkedin.com/in/vinni-aggarwal-b3a...,2025-06-17,What they don’t tell you about starting up: Yo...,https://www.linkedin.com/feed/update/urn:li:ac...,Yes,24,0,38
4,Daksh Goyal,Blinkit Complaint,https://www.linkedin.com/in/dakshg22?miniProfi...,2025-06-18,Day 1/100 – Finance Research Challenge Sector:...,https://www.linkedin.com/feed/update/urn:li:ac...,Yes,0,0,4


In [27]:
len(df)

1612

In [25]:
# df.to_csv("linkedin_posts_real.csv", index=False)